## Model Validation

**Course structure:**
* Chapter 1: Basic modeling in scikit-learn
* Chapter 2: Validation basics
* Chapter 3: Cross Validation
* Chapter 4: Selecting the best model with Hyperparameter tuning

### CHAPTER 1. Basic modeling in scikit-learn

#### 1.1 Introduction to model validation

**Model validation consists of:**
* Ensuring your model performs as expected on new data
* Testing model performance on holdout datasets
* Selecting the best model, parameters, and accuracy metrics
* Achieving the best accuracy for the data given

**Scikit-learn basic modeling review:**
* Creating a model by specifying the model type and parameters
* Fitting the model to training dataset
* Generating predictions for test dataset
* Looking at accuracy metrics

In [3]:
# seen vs. unseen data
import pandas as pd
candy_df = pd.read_csv('11_datasets/candy-data.csv')
print(candy_df.head())

ModuleNotFoundError: No module named 'pandas'